# Linear Solution

In [ ]:
class Linear(Node):
    def __init__(self, x, w, b):
        Node.__init__(self, [x, w, b])

    def forward(self):
        self.cache[0] = np.copy(self.input_nodes[0].value)
        self.cache[1] = np.copy(self.input_nodes[1].value)
        self.cache[2] = np.copy(self.input_nodes[2].value)
        self.value = np.dot(self.cache[0], self.cache[1]) + self.cache[2]

    def backward(self):
        self.dvalues = {n: np.zeros_like(n.value) for n in self.input_nodes}
        if len(self.output_nodes) == 0:
            self.dvalues[self.input_nodes[0]] += np.dot(np.ones_like(self.value), self.cache[1].T)
            self.dvalues[self.input_nodes[1]] += np.dot(self.cache[0].T, np.ones_like(self.value))
            self.dvalues[self.input_nodes[2]] += self.value.shape[0] # equivalent to summing this amount of 1s
            return
        for n in self.output_nodes:
            dval = n.dvalues[self]
            self.dvalues[self.input_nodes[0]] += np.dot(dval, self.cache[1].T)
            self.dvalues[self.input_nodes[1]] += np.dot(self.cache[0].T, dval)
            self.dvalues[self.input_nodes[2]] += np.sum(dval, axis=0, keepdims=True)

## Forward Pass

For the forward pass we compute

$$
XW + b
$$

and store the values in the cache.

## Backward Pass

Note the different initialization here, `np.zeros_like` creates an array the same shape as the input filled with 0s.

```
self.dvalues = {n: np.zeros_like(n.value) for n in self.input_nodes}
```

TODO: 